<a href="https://colab.research.google.com/github/pierre-brendan/PelotonIQ/blob/master/models/paris_nice_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# First attempt at a model to predict the winner of Paris - Nice

In [0]:
# Mount drive to get cyclist data
#from google.colab import drive
#drive.mount('/content/drive')

In [0]:
# All this code is from the Kmeans clustering code

# load modules
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import preprocessing

# load the rider data
cycling_data = pd.read_csv('/content/drive/My Drive/cycling_data/rider_data.csv')
cycling_data = cycling_data[cycling_data['Rider'] != 'Bjorg Lambrecht ']
cycling_data = cycling_data[cycling_data['Rider'] != 'Robbert de Greef ']

# Load the GC data
gc_winners = pd.read_csv('/content/drive/My Drive/cycling_data/Historical_GC_Classics_Results/gc_winners_consolidated.csv')
gc_winners.rename(columns = {'Cyclist':'Rider'}, inplace = True)

# Merge the GC winners to the rider data set
cycling_data = pd.merge(cycling_data, gc_winners, how='left', on='Rider')

# Load in the Monuments data
san_remo = pd.read_csv('/content/drive/My Drive/cycling_data/Historical_GC_Classics_Results/san_remo.csv')
san_remo = san_remo['san_remo_win'].groupby(san_remo['Rider']).sum()
roubaix = pd.read_csv('/content/drive/My Drive/cycling_data/Historical_GC_Classics_Results/roubaix.csv')
roubaix = roubaix['roubaix_win'].groupby(roubaix['Rider']).sum()
flanders = pd.read_csv('/content/drive/My Drive/cycling_data/Historical_GC_Classics_Results/flanders.csv')
flanders = flanders['flanders_win'].groupby(flanders['Rider']).sum()
lombardia = pd.read_csv('/content/drive/My Drive/cycling_data/Historical_GC_Classics_Results/lombardia.csv')
lombardia = lombardia['lombardia_win'].groupby(lombardia['Rider']).sum()
liege = pd.read_csv('/content/drive/My Drive/cycling_data/Historical_GC_Classics_Results/liege.csv')
liege = liege['lbl_win'].groupby(liege['Rider']).sum()
strade = pd.read_csv('/content/drive/My Drive/cycling_data/Historical_GC_Classics_Results/strade.csv')
strade['strade_win'] = 1
strade = strade['strade_win'].groupby(strade['Rider']).sum()
suisse = pd.read_csv('/content/drive/My Drive/cycling_data/Historical_GC_Classics_Results/suisse.csv')
suisse['suisse_win'] = 1
suisse = suisse['suisse_win'].groupby(suisse['Rider']).sum()
romandie = pd.read_csv('/content/drive/My Drive/cycling_data/Historical_GC_Classics_Results/romandie.csv')
romandie['romandie_win'] = 1
romandie = romandie['romandie_win'].groupby(romandie['Rider']).sum()
paris_nice = pd.read_csv('/content/drive/My Drive/cycling_data/Historical_GC_Classics_Results/paris_nice.csv')
paris_nice['paris_nice_win'] = 1
paris_nice = paris_nice['paris_nice_win'].groupby(paris_nice['Rider']).sum()
gent = pd.read_csv('/content/drive/My Drive/cycling_data/Historical_GC_Classics_Results/gent.csv')
gent['gent_win'] = 1
gent = gent['gent_win'].groupby(gent['Rider']).sum()
fleche = pd.read_csv('/content/drive/My Drive/cycling_data/Historical_GC_Classics_Results/fleche.csv')
fleche['fleche_win'] = 1
fleche = fleche['fleche_win'].groupby(fleche['Rider']).sum()
e3_binckbank = pd.read_csv('/content/drive/My Drive/cycling_data/Historical_GC_Classics_Results/e3_binckbank.csv')
e3_binckbank['e3_binckbank_win'] = 1
e3_binckbank = e3_binckbank['e3_binckbank_win'].groupby(e3_binckbank['Rider']).sum()
dauphine = pd.read_csv('/content/drive/My Drive/cycling_data/Historical_GC_Classics_Results/dauphine.csv')
dauphine['dauphine_win'] = 1
dauphine = dauphine['dauphine_win'].groupby(dauphine['Rider']).sum()
amstel = pd.read_csv('/content/drive/My Drive/cycling_data/Historical_GC_Classics_Results/amstel.csv')
amstel['amstel_win'] = 1
amstel = amstel['amstel_win'].groupby(amstel['Rider']).sum()


# Merge the Monuments winners to the rider data set
cycling_data = pd.merge(cycling_data, san_remo, how='left', on='Rider')
cycling_data = pd.merge(cycling_data, flanders, how='left', on='Rider')
cycling_data = pd.merge(cycling_data, liege, how='left', on='Rider')
cycling_data = pd.merge(cycling_data, lombardia, how='left', on='Rider')
cycling_data = pd.merge(cycling_data, roubaix, how='left', on='Rider')
cycling_data = pd.merge(cycling_data, strade, how='left', on='Rider')
cycling_data = pd.merge(cycling_data, suisse, how='left', on='Rider')
cycling_data = pd.merge(cycling_data, fleche, how='left', on='Rider')
cycling_data = pd.merge(cycling_data, gent, how='left', on='Rider')
cycling_data = pd.merge(cycling_data, paris_nice, how='left', on='Rider')
cycling_data = pd.merge(cycling_data, e3_binckbank, how='left', on='Rider')
cycling_data = pd.merge(cycling_data, amstel, how='left', on='Rider')
cycling_data = pd.merge(cycling_data, dauphine, how='left', on='Rider')
cycling_data = pd.merge(cycling_data, romandie, how='left', on='Rider')

# Replace NaN with 0's
cycling_data.fillna(0, inplace=True)

# Drop country variable
cycling_data = cycling_data.drop(['Country', 'crawl_date'], axis = 1)

In [0]:
#cycling_data.describe()
# Okay, let's make a column for winners of Paris-Nice type races
# Currently I'm thinking Paris - Nice, Tour de Suisse and Dauphine (Tierrento is not in the data set currently)
cycling_data['Paris_Nice_etc'] = cycling_data['paris_nice_win'] + cycling_data['dauphine_win'] + cycling_data['suisse_win']

# Will need to drop those 3 columns now
cycling_data = cycling_data.drop(['paris_nice_win', 'dauphine_win', 'suisse_win'], axis = 1)

# Split the data sets
import numpy as np
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(cycling_data, test_size = 0.1, 
                                       random_state = 42)
# first let's make a fresh copy of our data and drop what we are trying
# to predict so it doesn't have any transmations applied to it
psg = train_set.drop('Paris_Nice_etc', axis=1)
psg_labels = train_set['Paris_Nice_etc'].copy()
psg_rider = train_set['Rider'].copy()


In [0]:
# Also from KMeans code
# This would be the best approach to removing special characters
#dictionary = {'ž':'z', 'č':'c', 'ü':'u', 'á':'a', 'é':'e', 'ö':'o', 'ó':'o', 'ç':'c',
#              'ń':'ń', 'í':'i', 'è':'e'}
#cycling_data.replace(dictionary, regex=True, inplace=True)

#d = psg.drop(['Rider'], axis=1) #, 'Team'

# let's normalize the data
psg['GC'] = preprocessing.scale(psg['GC'])
psg['Sprint'] = preprocessing.scale(psg['Sprint'])
psg['TT'] = preprocessing.scale(psg['TT'])
psg['Climber'] = preprocessing.scale(psg['Climber'])
psg['Classic'] = preprocessing.scale(psg['Classic'])
psg['Age'] = preprocessing.scale(psg['Age'])
psg['Giro'] = preprocessing.scale(psg['Giro'])
psg['Vuelta'] = preprocessing.scale(psg['Vuelta'])
psg['Tour'] = preprocessing.scale(psg['Tour'])
psg['Total'] = preprocessing.scale(psg['Total'])
psg['san_remo_win'] = preprocessing.scale(psg['san_remo_win'])
psg['lombardia_win'] = preprocessing.scale(psg['lombardia_win'])
psg['flanders_win'] = preprocessing.scale(psg['flanders_win'])
psg['lbl_win'] = preprocessing.scale(psg['lbl_win'])
psg['roubaix_win'] = preprocessing.scale(psg['roubaix_win'])
psg['strade_win'] = preprocessing.scale(psg['strade_win'])
#psg['suisse_win'] = preprocessing.scale(psg['suisse_win'])
psg['fleche_win'] = preprocessing.scale(psg['fleche_win'])
psg['gent_win'] = preprocessing.scale(psg['gent_win'])
#psg['Paris_Nice_etc'] = preprocessing.scale(psg['Paris_Nice_etc'])
psg['e3_binckbank_win'] = preprocessing.scale(psg['e3_binckbank_win'])
psg['amstel_win'] = preprocessing.scale(psg['amstel_win'])
#psg['dauphine_win'] = preprocessing.scale(psg['dauphine_win'])
psg['romandie_win'] = preprocessing.scale(psg['romandie_win'])

# Algorithims prefer working with numbers generally, so let's convert the team
# proximity columns to numeric values
psg['Team'] = psg['Team'].astype(str)
housing_cat = psg[['Team']]
# Problem is if we use numeric examples for these variables algorithims
# will think they are near eachother. Instead we should make some one-hot
# encodings otherwise known as dummy variables for each category as it's own
# column
from sklearn.preprocessing import OrdinalEncoder
#ordinal_encoder = OrdinalEncoder()
from sklearn.preprocessing import OneHotEncoder
#cat_encoder = OrdinalEncoder()
#psg_cat_1hot = cat_encoder.fit_transform(housing_cat)

from sklearn.compose import ColumnTransformer
#cat_attribs = ['Team']

#full_pipeline = ColumnTransformer([
#      ('cat', OneHotEncoder(), cat_attribs),
#])

#psg_prepared = full_pipeline.fit_transform(psg) # applying this to pandas dataframe called housing

# not a fan of the approach above, changing it in the section below


In [0]:
# use pd.concat to join the new columns with your original dataframe
psg = pd.concat([psg,pd.get_dummies(psg['Team'], prefix='Team')],axis=1)

# now drop the original 'country' column (you don't need it anymore)
psg.drop(['Team'],axis=1, inplace=True)

,Rider,GC,TT,Sprint,Climber,Classic,Age,Giro,Tour,Vuelta,Total,san_remo_win,flanders_win,lbl_win,lombardia_win,roubaix_win,strade_win,fleche_win,gent_win,e3_binckbank_win,amstel_win,romandie_win,Team_ Israel Cycling Academy,Team_0,Team_7 Eleven Cliqq - air21 by Roadbike Philippines,Team_AG2R La Mondiale,Team_Adria Mobil,Team_Aisan Racing Team,Team_Alpecin-Fenix,Team_Amore & Vita - Prodir,Team_Androni Giocattoli - Sidermec,Team_Astana Pro Team,Team_Atum general / Tavira / Maria Nova Hotel,Team_Aviludo-Louletano,Team_B&B Hotels - Vital Concept p/b KTM,Team_BEAT Cycling Club,Team_BHS - PL Beton Bornholm,Team_BORA - hansgrohe,Team_Bahrain - McLaren,Team_Bahrain Cycling Academy,...,Team_Rally Cycling,Team_Riwal Readynez Cycling Team,Team_SEG Racing Academy,Team_SSIOS Miogee Cycling Team,Team_Shenzhen Xidesheng Cycling Team,Team_Sport Vlaanderen - Baloise,Team_St George Continental Cycling Team,Team_St Michel - Auber93,Team_Swiss Racing Academy,Team_Tarteletto - Isorex,Team_Team Arkéa Samsic,Team_Team BridgeLane,Team_Team Bridgestone Cycling,Team_Team Felbermayr - Simplon Wels,Team_Team Hrinkow Advarics Cycleang,Team_Team INEOS,Team_Team Jumbo-Visma,Team_Team Medellin,Team_Team Novo Nordisk,Team_Team Sapura Cycling,Team_Team Skyline,Team_Team Sunweb,Team_Team Total Direct Energie,Team_Team UKYO,Team_Team Vorarlberg Santic,Team_Terengganu Inc. TSG Cycling Team,Team_Tianyoude Hotel Cycling Team,Team_Trek - Segafredo,Team_UAE-Team Emirates,Team_Uno-X Norwegian Development Team,Team_Utsunomiya Blitzen,Team_Vini Zabù - KTM,Team_Vino - Astana Motors,Team_VolkerWessels-Merckx,Team_Voster ATS Team,Team_W52 / FC Porto,Team_WSA KTM Graz,Team_Wibatech Merx 7R,Team_Work Service Dynatek Vega,Team_À BLOC CT
428,Pim Ligthart,-0.022062,-0.373734,0.430120,-0.288801,0.406918,0.734329,-0.079416,-0.06004,-0.094836,-0.089671,-0.091799,-0.083741,-0.06004,-0.079012,-0.083741,-0.079416,-0.05459,-0.072645,-0.091799,-0.0764,-0.079012,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
137,Luka Mezgec,-0.326809,-0.407047,1.705693,-0.318085,0.307252,0.734329,-0.079416,-0.06004,-0.094836,-0.089671,-0.091799,-0.083741,-0.06004,-0.079012,-0.083741,-0.079416,-0.05459,-0.072645,-0.091799,-0.0764,-0.079012,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
72,Dan Martin,3.990579,0.138986,-0.057217,4.960114,2.223250,1.204038,-0.079416,-0.06004,-0.094836,-0.089671,-0.091799,-0.083741,-0.06004,-0.079012,-0.083741,-0.079416,-0.05459,-0.072645,-0.091799,-0.0764,-0.079012,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
77,Alberto Bettiol,-0.331989,-0.199931,-0.301228,-0.194738,0.481668,-0.439943,-0.079416,-0.06004,-0.094836,-0.089671,-0.091799,11.941524,-0.06004,-0.079012,-0.083741,-0.079416,-0.05459,-0.072645,-0.091799,-0.0764,-0.079012,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
512,Nathan Van Hooydonck,-0.444219,-0.155031,-0.501371,-0.450306,-0.488360,-0.909653,-0.079416,-0.06004,-0.094836,-0.089671,-0.091799,-0.083741,-0.06004,-0.079012,-0.083741,-0.079416,-0.05459,-0.072645,-0.091799,-0.0764,-0.079012,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
# Drop rider name
psg = psg.drop(['Rider'], axis=1)

# rename so rest of coding works
psg_prepared = psg

# Load this for RMSE calcs
from sklearn.metrics import mean_squared_error

# Linear regression
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(psg_prepared, psg_labels)

# Let's test more powerful model
from sklearn.tree import DecisionTreeRegressor
# Build decision tree model
tree_reg = DecisionTreeRegressor()
tree_reg.fit(psg_prepared, psg_labels)

# we use 10 folds, which splits the data into 10 random folds
# and evaluates the decision tree model 10 times
# we test against a negative score cross-validation features expect
# a utility function (greater is better) rather than a cost function
# (where lower is better), this is the opposite of MSE
from sklearn.model_selection import cross_val_score
scores = cross_val_score(tree_reg, psg_prepared, psg_labels,
                         scoring='neg_mean_squared_error', cv=10)
tree_rmse_scores = np.sqrt(-scores)

# LEt's look at the results
def display_scores(scores):
  print('Scores:', scores)
  print('Mean:', scores.mean())
  print('Standard Deviation:', scores.std())

display_scores(tree_rmse_scores)  


Scores: [0.         0.372678   0.         0.33333333 0.23570226 0.23570226
 0.11785113 0.26352314 0.56916049 0.44405304]
Mean: 0.25720036434718807
Standard Deviation: 0.17437056253982827


In [0]:
# let's compute the scores for the linear regression to be sure
lin_scores = cross_val_score(lin_reg, psg_prepared, psg_labels,
                             scoring = 'neg_mean_squared_error', cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)
# Woah, this is terrible after adding in the team dummy variables
# will need unique data set if using LR

Scores: [1.24388453e+09 7.12254774e+07 3.16031423e+08 4.87153973e+08
 1.82874838e+08 1.90755217e+10 1.04296014e+09 9.59344084e+10
 2.61167117e+08 9.23662488e+07]
Mean: 11870759385.990467
Standard Deviation: 28567856799.1592


In [0]:
# Let's try a random forest model
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor()
forest_reg.fit(psg_prepared, psg_labels)
psg_predictions = forest_reg.predict(psg_prepared)
forest_mse = mean_squared_error(psg_labels, psg_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse
scores = cross_val_score(forest_reg, psg_prepared, psg_labels,
                         scoring='neg_mean_squared_error', cv=10)
forest_rmse_scores = np.sqrt(-scores)
display_scores(forest_rmse_scores)

Scores: [0.14359472 0.20994047 0.01118034 0.20273135 0.08957461 0.22330224
 0.14158331 0.29971051 0.34843918 0.37725472]
Mean: 0.20473114454300578
Standard Deviation: 0.10874821245577707


In [0]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {
        'n_estimators': randint(low=75, high=100),
        'max_features': randint(low=75, high=110),
    }

forest_reg = RandomForestRegressor(random_state=42)
rnd_search = RandomizedSearchCV(forest_reg, param_distributions=param_distribs,
                                n_iter=30, cv=5, scoring='neg_mean_squared_error', random_state=42)
rnd_search.fit(psg_prepared, psg_labels)
# grid search can take a while, this is a randomized approach

RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   ccp_alpha=0.0,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=100,
                              

In [0]:
cvres = rnd_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

0.23006142871589655 {'max_features': 103, 'n_estimators': 89}
0.23240589684396382 {'max_features': 82, 'n_estimators': 95}
0.23034173108009565 {'max_features': 93, 'n_estimators': 97}
0.2321555292013315 {'max_features': 85, 'n_estimators': 85}
0.23056398025745253 {'max_features': 98, 'n_estimators': 95}
0.22743361328040157 {'max_features': 98, 'n_estimators': 77}
0.23072815333122196 {'max_features': 96, 'n_estimators': 95}
0.22822788414771839 {'max_features': 76, 'n_estimators': 98}
0.23103767434402897 {'max_features': 104, 'n_estimators': 80}
0.22742121225933332 {'max_features': 76, 'n_estimators': 95}
0.2308947965002987 {'max_features': 107, 'n_estimators': 86}
0.2300699570685919 {'max_features': 96, 'n_estimators': 86}
0.2345732293808958 {'max_features': 99, 'n_estimators': 91}
0.2322770146654577 {'max_features': 101, 'n_estimators': 84}
0.23025922068209023 {'max_features': 102, 'n_estimators': 90}
0.2291790684494485 {'max_features': 89, 'n_estimators': 89}
0.22471921230529476 {'max

In [0]:
# LEt's go back to the default
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor()
forest_reg.fit(psg_prepared, psg_labels)
psg_predictions = forest_reg.predict(psg_prepared)
forest_mse = mean_squared_error(psg_labels, psg_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

0.09219091248818344

In [0]:
# let's look at the relative importance of each attribute
feature_importances = forest_reg.feature_importances_
feature_importances
attributes = list(psg_prepared)
sorted(zip(feature_importances, attributes), reverse=True)

In [0]:
# let's evaluate the model on the test set
final_model = forest_reg
x_test = test_set.drop('Paris_Nice_etc', axis=1)
y_test = test_set['Paris_Nice_etc'].copy()
test_rider = test_set['Rider'].copy()

########
# Adjust the data
# let's normalize the data
x_test['GC'] = preprocessing.scale(x_test['GC'])
x_test['Sprint'] = preprocessing.scale(x_test['Sprint'])
x_test['TT'] = preprocessing.scale(x_test['TT'])
x_test['Climber'] = preprocessing.scale(x_test['Climber'])
x_test['Classic'] = preprocessing.scale(x_test['Classic'])
x_test['Age'] = preprocessing.scale(x_test['Age'])
x_test['Giro'] = preprocessing.scale(x_test['Giro'])
x_test['Vuelta'] = preprocessing.scale(x_test['Vuelta'])
x_test['Tour'] = preprocessing.scale(x_test['Tour'])
x_test['Total'] = preprocessing.scale(x_test['Total'])
x_test['san_remo_win'] = preprocessing.scale(x_test['san_remo_win'])
x_test['lombardia_win'] = preprocessing.scale(x_test['lombardia_win'])
x_test['flanders_win'] = preprocessing.scale(x_test['flanders_win'])
x_test['lbl_win'] = preprocessing.scale(x_test['lbl_win'])
x_test['roubaix_win'] = preprocessing.scale(x_test['roubaix_win'])
x_test['strade_win'] = preprocessing.scale(x_test['strade_win'])
#x_test['suisse_win'] = preprocessing.scale(x_test['suisse_win'])
x_test['fleche_win'] = preprocessing.scale(x_test['fleche_win'])
x_test['gent_win'] = preprocessing.scale(x_test['gent_win'])
#x_test['Paris_Nice_etc'] = preprocessing.scale(x_test['Paris_Nice_etc'])
x_test['e3_binckbank_win'] = preprocessing.scale(x_test['e3_binckbank_win'])
x_test['amstel_win'] = preprocessing.scale(x_test['amstel_win'])
#x_test['dauphine_win'] = preprocessing.scale(x_test['dauphine_win'])
x_test['romandie_win'] = preprocessing.scale(x_test['romandie_win'])

# Algorithims prefer working with numbers generally, so let's convert the team
# proximity columns to numeric values
x_test['Team'] = x_test['Team'].astype(str)

# use pd.concat to join the new columns with your original dataframe
x_test = pd.concat([x_test,pd.get_dummies(x_test['Team'], prefix='Team')],axis=1)

# now drop the original 'country' column (you don't need it anymore)
x_test.drop(['Team'],axis=1, inplace=True)

# Drop rider name
x_test = x_test.drop(['Rider'], axis=1)

########

x_test_prepared = x_test
final_predictions = final_model.predict(x_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)